In [15]:
import pandas as pd
import numpy as np

# Load the cleaned datasets
client_df = pd.read_csv('clean_data_after_eda.csv')
price_df = pd.read_csv('price_data.csv')

price_df['price_date'] = pd.to_datetime(price_df['price_date'])

# Aggregate price data to extract Jan and Dec values per Customer ID
jan_prices = price_df[price_df['price_date'].dt.month == 1]
dec_prices = price_df[price_df['price_date'].dt.month == 12]

# Join January and December prices
price_features = pd.merge(
    jan_prices[['id', 'price_off_peak_var', 'price_off_peak_fix']],
    dec_prices[['id', 'price_off_peak_var', 'price_off_peak_fix']],
    on='id',
    suffixes=('_jan', '_dec')
)

# Calculate the annual price difference
price_features['offpeak_diff_dec_jan_var'] = price_features['price_off_peak_var_dec'] - price_features['price_off_peak_var_jan']
price_features['offpeak_diff_dec_jan_fix'] = price_features['price_off_peak_fix_dec'] - price_features['price_off_peak_fix_jan']

# Merge these volatility features into the client data
client_df = pd.merge(client_df, price_features[['id', 'offpeak_diff_dec_jan_var', 'offpeak_diff_dec_jan_fix']], on='id', how='left')

date_cols = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal']
for col in date_cols:
    client_df[col] = pd.to_datetime(client_df[col])

# Feature: Tenure in years (Reference date Jan 2016)
ref_date = pd.Timestamp(2016, 1, 1)
client_df['tenure_years'] = (ref_date - client_df['date_activ']).dt.days / 365.25

# Feature: Months since last product modification
client_df['months_since_modification'] = ((ref_date - client_df['date_modif_prod']).dt.days / 30).astype(int)

# Feature: Total yearly consumption (Elec + Gas)
client_df['total_cons_12m'] = client_df['cons_12m'] + client_df['cons_gas_12m']

# Feature: Consumption volatility (Last month usage vs average monthly usage)
client_df['cons_volatility'] = client_df['cons_last_month'] / (client_df['cons_12m'] / 12 + 1e-6)

# Feature: Forecasted Price Spread (Peak - Off-Peak)
client_df['forecast_price_spread'] = client_df['forecast_price_energy_peak'] - client_df['forecast_price_energy_off_peak']

# Apply log(1+x) to highly skewed consumption variables for model stability
skewed_cols = ['cons_12m', 'cons_gas_12m', 'cons_last_month', 'forecast_cons_12m', 'total_cons_12m']
for col in skewed_cols:
    client_df[f'log_{col}'] = np.log1p(client_df[col].clip(lower=0))

# Final Clean-up: Remove raw date columns
client_df_final = client_df.drop(columns=date_cols)

# Save the final engineered dataset
#client_df_final.to_csv('final_engineered_data.csv', index=False)